<a href="https://colab.research.google.com/github/Lee-gp/classification/blob/main/%E5%91%98%E5%B7%A5%E7%A6%BB%E8%81%8C%E7%8E%87%E9%A2%84%E6%B5%8B_stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import time,datetime
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn import metrics
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#数据加载
from google.colab import drive
import os
drive.mount("/content/drive")
os.chdir("/content/drive/My Drive/Colab Notebooks/BI_core_L1")

train = pd.read_csv('./lz_train.csv')
test = pd.read_csv('./lz_test.csv')
#准备提交数据
submit2 = pd.DataFrame(test['user_id'])
#查看标签
print(train['Attrition'].value_counts())
#标签字段labelEncoder
train['Attrition']= train['Attrition'].map(lambda x:1 if x == 'Yes' else 0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No     988
Yes    188
Name: Attrition, dtype: int64


In [3]:
#查看是否有空值
# print(train.isna().sum())
# train.head()

In [4]:
#查看特征取值情况
# for c in train.columns:
#     print(train[c].value_counts())

In [5]:
#去掉没有用的列，员工编号，员工工时,大于18岁,员工数量
train = train.drop(['EmployeeNumber','StandardHours','Over18','EmployeeCount','user_id'],axis = 1)
test = test.drop(['EmployeeNumber','StandardHours','Over18','EmployeeCount','user_id'],axis = 1)

In [6]:
#查看数据情况
# train.info()
# test.info()

In [7]:
#类别列
cat_cols = ['BusinessTravel',
            'Department',
            'DistanceFromHome',          
            'Education',
            'EducationField', 
            'EnvironmentSatisfaction',
            'Gender',
            'JobInvolvement',
            'JobLevel',
            'JobRole',
            'JobSatisfaction',
            'MaritalStatus',           
            'OverTime',            
            'PerformanceRating',
            'RelationshipSatisfaction',
            'StockOptionLevel',
            'TrainingTimesLastYear',
            'WorkLifeBalance',
            'NumCompaniesWorked',
            'YearsInCurrentRole'    
            ]

In [8]:
#数值列
digital_col = ['Age',
                'DailyRate',          
                'HourlyRate',
                 'MonthlyIncome',
                  'MonthlyRate',           
                    'PercentSalaryHike',          
               'TotalWorkingYears',
                  'YearsAtCompany', 
              'YearsSinceLastPromotion',
                  'YearsWithCurrManager'    
               ]

In [9]:
#使用catboost训练
# !pip install catboost
from catboost import CatBoostClassifier,Pool

labels = train['Attrition']
features = train.drop(['Attrition'],axis =1)
#划分数据集
X_train,X_valid,y_train,y_valid = train_test_split(features,labels,test_size = 0.2,random_state = 9)
#训练数据
train_data = Pool(data = X_train,
                 label = y_train,
                 cat_features = cat_cols)
#构建模型
clf = CatBoostClassifier(depth =10,
                         loss_function = 'Logloss',
                        bootstrap_type= 'Bernoulli',
                        subsample = 0.8,
                        learning_rate = 0.01,
                        l2_leaf_reg=6,
                        iterations = 1400,                     
                        rsm =0.8,
                        od_type = 'Iter',
                        od_wait =200,
                        metric_period =200,
                        thread_count = 10,
                        random_seed =9,
                        one_hot_max_size =30,
                        eval_metric = 'AUC:hints=skip_train~false',
                        custom_metric = 'AUC',
                        #scale_pos_weight = 3,
                         #grow_policy = 'Lossguide'
                        )
#模型训练
clf.fit(train_data,eval_set =(X_valid,y_valid))
#预测
y_hat = clf.predict_proba(X_valid)[:,1]

print('预测auc',roc_auc_score(list(y_valid),y_hat))

0:	learn: 0.6688456	test: 0.6104440	best: 0.6104440 (0)	total: 53.4ms	remaining: 1m 14s
200:	learn: 0.9964657	test: 0.8308943	best: 0.8335210 (177)	total: 3.08s	remaining: 18.4s
400:	learn: 0.9999400	test: 0.8388993	best: 0.8393996 (397)	total: 6.23s	remaining: 15.5s
600:	learn: 1.0000000	test: 0.8421513	best: 0.8421513 (599)	total: 9.31s	remaining: 12.4s
800:	learn: 1.0000000	test: 0.8387742	best: 0.8422764 (602)	total: 12.5s	remaining: 9.32s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8422764228
bestIteration = 602

Shrink model to first 603 iterations.
预测auc 0.8422764227642277


In [10]:
#将预测结果作为下一步训练的特征
new_train_feature = clf.predict_proba(features)[:,1].reshape(-1,1)
new_test_feature = clf.predict_proba(test)[:,1].reshape(-1,1)

In [11]:
#print(new_test_feature)
print(new_train_feature.shape)

(1176, 1)


In [13]:
# 对分类特征进行编码
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

train_cat,test_cat = None,None
for feature in cat_cols:
    lbe,ohe = LabelEncoder(),OneHotEncoder()    
    
    lbe.fit(pd.concat([train[feature],test[feature]]).values.reshape(-1,1))
    train[feature] = lbe.transform(train[feature])
    test[feature] = lbe.transform(test[feature]) 
    
    #print(type(pd.concat([train[feature],test[feature]]).values.reshape(-1,1)))  
    
    ohe.fit(pd.concat([train[feature],test[feature]]).values.reshape(-1,1))
      
#     train = train.join(pd.get_dummies(train[feature],prefix = feature)).drop([feature],axis =1)
#     test = test.join(pd.get_dummies(test[feature],prefix = feature)).drop([feature],axis =1)
    oht_train = ohe.transform(train[feature].values.reshape(-1,1)).todense()
    oht_test = ohe.transform(test[feature].values.reshape(-1,1)).todense()
    
    if train_cat is None:
        train_cat = oht_train
        test_cat = oht_test
    else:
        train_cat = np.hstack((train_cat,oht_train))
        test_cat = np.hstack((test_cat,oht_test))    

# train.info()
# train.head()

In [14]:
#对数字类特征进行规范化
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_digital = pd.DataFrame(scaler.fit_transform(train[digital_col]),columns = digital_col)
test_digital = pd.DataFrame(scaler.transform(test[digital_col]),columns=digital_col)

label = train['Attrition']
train = pd.DataFrame(np.hstack((train_digital,train_cat)))
test = pd.DataFrame(np.hstack((test_digital,test_cat)))
# train.info()

In [15]:
#切分数据集
X_train,X_valid,y_train,y_valid = train_test_split(train,label,test_size = 0.2,random_state = 9)

In [16]:
#使用xgboost训练
import xgboost as xgb
from xgboost import XGBClassifier

In [17]:
param_xgb = {'booster':'gbtree',
        'objective':'binary:logistic',
        'eval_metric':'auc',
        'eta':0.01,
        'max_depth':6,
        'colsample_bytree':0.8,
        'colsample_bylevel':0.8, 
         'subsample':0.8,
         'alpha':0.6,
        'lambda':0,
         'min_child_weight':1,
         'gamma':0,
         'tree_method':'exact',
         #'scale_pos_weight':0.18         
        }

In [18]:
train_data = xgb.DMatrix(X_train,label = y_train)
valid_data = xgb.DMatrix(X_valid,label = y_valid)
test_data = xgb.DMatrix(test)

model_xgb = xgb.train(param_xgb,
            train_data,evals = [(train_data,'train'),(valid_data,'valid')],
            num_boost_round = 10000,
            early_stopping_rounds = 200,
            verbose_eval=200)


[0]	train-auc:0.832686	valid-auc:0.70763
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 200 rounds.
[200]	train-auc:0.987158	valid-auc:0.794246
[400]	train-auc:0.998799	valid-auc:0.818637
[600]	train-auc:0.99994	valid-auc:0.825266
[800]	train-auc:1	valid-auc:0.828018
[1000]	train-auc:1	valid-auc:0.831895
[1200]	train-auc:1	valid-auc:0.832395
[1400]	train-auc:1	valid-auc:0.833521
[1600]	train-auc:1	valid-auc:0.83252
Stopping. Best iteration:
[1467]	train-auc:1	valid-auc:0.834146



In [19]:
#将xgboost的输出作为下一步训练的输入
xgb_data = xgb.DMatrix(train)
new_train_feature = np.hstack((new_train_feature,model_xgb.predict(xgb_data).reshape(-1,1)))
new_test_feature =  np.hstack((new_test_feature,model_xgb.predict(test_data).reshape(-1,1)))

In [20]:
# print(new_test_feature)

In [21]:
#使用GBDT进行训练
from sklearn.ensemble import GradientBoostingClassifier

gbdt = GradientBoostingClassifier(#loss='exponential',
    learning_rate=0.02,
    n_estimators=1000,
    subsample=0.7,
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    init=None,
    random_state=10,
    max_features=None,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    presort='deprecated',
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
    ccp_alpha=0,)
gbdt.fit(X_train,y_train)

gbdt_valid = gbdt.predict_proba(X_valid)[:,1]
print('预测auc',roc_auc_score(list(y_valid),gbdt_valid))

预测auc 0.8276422764227642


In [22]:
#将GBDT的输出作为下一步训练的输入
new_train_feature = np.hstack((new_train_feature,gbdt.predict_proba(train)[:,1].reshape(-1,1)))
new_test_feature =  np.hstack((new_test_feature,gbdt.predict_proba(test)[:,1].reshape(-1,1)))

In [25]:
print(train_cat.shape)

(1176, 129)


In [26]:
#使用lightgbm训练
import lightgbm as lgb

param_lgb = { 'boosting_type':'gbdt',
    'num_leaves':15,
    'max_depth':4,
    'learning_rate':0.01,
    'n_estimators':10000,
    'subsample_for_bin':200000,
    'objective':'binary',
    'scale_pos_weight':0.8,
    #'min_split_gain':0.0,
    #'min_child_weight':0.001,
    #'min_child_samples':20,
    #'subsample':1.0,
    #'subsample_freq':0,
    #'colsample_bytree':1.0,
    #'reg_alpha':0.0,
    #'reg_lambda':0.0,
    'random_state':9,
    'n_jobs':10,
    #'silent':True,
    #'importance_type':'split', 
    'metric':'auc',
    'feature_fraction':0.9,
    'bagging_fraction':0.8,
    'bagging_freq':8,
    'lambda_l1':0.5,
    'lambda_l2':0.1
        }
train_data = lgb.Dataset(X_train,label = y_train)
valid_data = lgb.Dataset(X_valid,label = y_valid)

# lgb_data = lgb.Dataset(train)
# test_data = lgb.Dataset(test)


lgb = lgb.train(param_lgb,
         train_data,
        valid_sets = [train_data,valid_data],
        num_boost_round = 10000,
        early_stopping_rounds =200,
        verbose_eval =200,
        # categorical_feature = pd.DataFrame(train_cat).columns
        )


Training until validation scores don't improve for 200 rounds.
[200]	training's auc: 0.934598	valid_1's auc: 0.787367
[400]	training's auc: 0.966115	valid_1's auc: 0.812383
[600]	training's auc: 0.979369	valid_1's auc: 0.815635
[800]	training's auc: 0.990735	valid_1's auc: 0.82339
[1000]	training's auc: 0.996757	valid_1's auc: 0.822639
[1200]	training's auc: 0.998696	valid_1's auc: 0.829518
Early stopping, best iteration is:
[1160]	training's auc: 0.998379	valid_1's auc: 0.831144


In [27]:
#将lightgbm的输出作为下一步训练的输入
new_train_feature = np.hstack((new_train_feature,lgb.predict(train).reshape(-1,1)))
new_test_feature =  np.hstack((new_test_feature,lgb.predict(test).reshape(-1,1)))

In [28]:
#使用svm预测
from sklearn.svm import SVC

svc = SVC(decision_function_shape = 'ovr',
          C=0.7,
          kernel = 'rbf',
          #degree = 2,
          coef0 = 0.8,
         gamma = 'auto',
         max_iter = 148888,
         random_state = 33,
         verbose = True,
         tol = 1e-5,
         cache_size = 50000,
         probability = True,
         class_weight = 'balanced'
         )

svc.fit(X_train,y_train)

svc_valid = svc.predict_proba(X_valid)[:,1]
#print(svc_valid)
print('预测auc',roc_auc_score(list(y_valid),svc_valid))


[LibSVM]预测auc 0.8197623514696686


In [29]:
#将svc的预测结果作为下一步训练的特征
new_train_feature = np.hstack((new_train_feature,svc.predict_proba(train)[:,1].reshape(-1,1)))
new_test_feature =  np.hstack((new_test_feature,svc.predict_proba(test)[:,1].reshape(-1,1)))

In [30]:
#使用KNN进行预测
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 9)

knn.fit(X_train,y_train)

knn_valid = knn.predict_proba(X_valid)[:,1]
print('预测auc:',roc_auc_score(list(y_valid),knn_valid))

预测auc: 0.7502188868042527


In [31]:
#使用朴素贝叶斯预测
from sklearn.naive_bayes import BernoulliNB

nb = BernoulliNB(alpha = 17, fit_prior = True,binarize =0.38)

nb.fit(X_train,y_train)

nb_valid = nb.predict_proba(X_valid)[:,1]
print('预测auc:',roc_auc_score(list(y_valid),nb_valid))

预测auc: 0.8381488430268919


In [32]:
#将朴素贝叶斯的输出作为下一步训练的特征
new_train_feature = np.hstack((new_train_feature,nb.predict_proba(train)[:,1].reshape(-1,1)))
new_test_feature =  np.hstack((new_test_feature,nb.predict_proba(test)[:,1].reshape(-1,1)))

In [33]:
#使用Adaboost训练
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators =98,
                        learning_rate = 0.2,
                        algorithm = 'SAMME.R',
                        random_state =1)

ada.fit(X_train,y_train)

ada_valid = ada.predict_proba(X_valid)[:,1]
print('预测auc:',roc_auc_score(list(y_valid),ada_valid))

预测auc: 0.8327704815509693


In [34]:
#将Adaboost的输出作为下一步训练的特征
new_train_feature = np.hstack((new_train_feature,ada.predict_proba(train)[:,1].reshape(-1,1)))
new_test_feature =  np.hstack((new_test_feature,ada.predict_proba(test)[:,1].reshape(-1,1)))

In [35]:
#使用随机森林进行训练
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion = 'gini',
                           n_estimators = 99,
                           max_depth =11,
                           min_samples_split = 6,
                           min_samples_leaf = 1,
                           max_features = 'auto',
                           oob_score = True,
                           random_state = 1,
                           class_weight = 'balanced_subsample')
rf.fit(X_train,y_train)

rf_valid = rf.predict_proba(X_valid)[:,1]
print('预测auc:',roc_auc_score(list(y_valid),rf_valid))

预测auc: 0.8142589118198874


In [36]:
#将随机森林的输出作为下一步训练的特征
new_train_feature = np.hstack((new_train_feature,rf.predict_proba(train)[:,1].reshape(-1,1)))
new_test_feature =  np.hstack((new_test_feature,rf.predict_proba(test)[:,1].reshape(-1,1)))

In [37]:
#查看训练数据情况
print(type(new_train_feature))
print(new_train_feature.shape)
print(new_train_feature)

<class 'numpy.ndarray'>
(1176, 8)
[[5.10374092e-02 1.05480012e-02 1.31622970e-02 ... 1.01859569e-03
  4.32903114e-01 1.02231174e-01]
 [2.47078596e-02 6.97884196e-03 7.73743298e-03 ... 1.92954868e-04
  4.57661850e-01 8.73663085e-02]
 [6.20154293e-02 4.59669195e-02 5.28928711e-02 ... 3.27380754e-04
  4.64740365e-01 2.12177422e-01]
 ...
 [4.10868075e-02 7.46613555e-03 1.32504845e-02 ... 1.57277611e-04
  4.66636238e-01 8.22206684e-02]
 [1.12883712e-01 8.46080706e-02 1.72501948e-01 ... 2.32112551e-02
  4.86349671e-01 3.53981019e-01]
 [2.03773226e-01 6.32306337e-01 5.20117939e-01 ... 2.59042409e-03
  4.85085323e-01 3.62678294e-01]]


In [38]:
#使用LR模型作为stack输出
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(class_weight = 'balanced',tol=1e-5,C=7)
X = pd.DataFrame(new_train_feature)
y = label

X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size = 0.2,random_state = 42)

lr.fit(X_train,y_train)
lr_valid = lr.predict_proba(X_valid)[:,1]
print('预测auc',roc_auc_score(list(y_valid),lr_valid))

预测auc 0.9840663538142529


In [39]:
#生成提交数据
test_data = pd.DataFrame(new_test_feature)
y_hat = lr.predict_proba(test_data)[:,1]
predict = [round(x,4) for x in y_hat]
# print(predict)
submit2['Attrition'] = predict
submit2.to_csv('submission.csv', index=False)

[0.0751, 0.0445, 0.2131, 0.0728, 0.9999, 0.6788, 0.6391, 0.1113, 0.062, 0.1883, 0.06, 0.1256, 0.0879, 0.999, 0.0404, 0.0405, 0.0777, 0.0766, 0.0761, 0.0897, 0.9898, 0.0578, 0.0483, 0.0721, 0.2738, 0.9085, 0.0493, 0.0411, 0.9993, 0.0705, 0.0485, 0.1178, 0.131, 0.0763, 0.0773, 0.0587, 0.0539, 0.1032, 0.0499, 0.0932, 0.06, 0.0673, 0.0568, 0.0829, 0.0801, 0.9988, 0.1058, 0.037, 0.9979, 0.8181, 0.3345, 0.9517, 0.131, 0.0748, 0.9978, 0.151, 0.0503, 0.0698, 0.043, 0.9422, 0.0774, 0.1375, 0.0975, 0.0728, 0.8863, 0.1158, 0.1276, 0.0777, 0.0567, 0.2619, 0.0488, 0.2331, 0.0856, 0.055, 0.0553, 0.0607, 0.0706, 0.0736, 0.9544, 0.0452, 0.0447, 0.0599, 0.0884, 0.1269, 0.0832, 0.061, 0.0488, 0.3301, 0.0943, 0.0592, 0.9459, 0.0465, 0.1075, 0.9181, 0.0474, 0.0949, 0.1001, 0.9678, 0.0596, 0.0861, 0.2475, 0.1392, 0.8938, 0.0597, 0.058, 0.0515, 0.0742, 0.9692, 0.1095, 0.1424, 0.2141, 0.2323, 0.0559, 0.0663, 0.0828, 0.1593, 0.0492, 0.2466, 0.0492, 0.0354, 0.0563, 0.0442, 0.0502, 0.9972, 0.579, 0.0602, 0.0413